In [29]:
import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [30]:
#图片预处理
transform = transforms.Compose([
        transforms.Scale(36),
        transforms.RandomCrop(32),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

In [31]:
#CIFAR-10 数据集
train_dataset = dsets.CIFAR10(root='../data/',
                               train=True, 
                               transform=transform,
                               download=True)

Files already downloaded and verified


In [32]:
#加载数据集
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

In [33]:
#4x4 Convolution
def conv4x4(in_channels, out_channels, stride):
    return nn.Conv2d(in_channels, out_channels, kernel_size=4, 
                     stride=stride, padding=1, bias=False)

In [34]:
#构建判别网络
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            conv4x4(3, 16, 2),
            nn.LeakyReLU(0.2, inplace=True),
            conv4x4(16, 32, 2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, inplace=True),
            conv4x4(32, 64, 2), 
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 1, kernel_size=4),
            nn.Sigmoid())
    
    def forward(self, x):
        out = self.model(x)
        out = out.view(out.size(0), -1)
        return out

In [35]:
# 4x4 Transpose convolution
def conv_transpose4x4(in_channels, out_channels, stride=1, padding=1, bias=False):
    return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4, 
                              stride=stride, padding=padding, bias=bias)

In [36]:
#构建生成网络
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            conv_transpose4x4(128, 64, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            conv_transpose4x4(64, 32, 2),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            conv_transpose4x4(32, 16, 2),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            conv_transpose4x4(16, 3, 2, bias=True),
            nn.Tanh())
    
    def forward(self, x):
        x = x.view(x.size(0), 128, 1, 1)
        out = self.model(x)
        return out

In [37]:
discriminator = Discriminator()
generator = Generator()

Generator (
  (model): Sequential (
    (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU (inplace)
    (9): ConvTranspose2d(16, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): Tanh ()
  )
)

In [38]:
#定义损失函数、优化函数
criterion = nn.BCELoss()
lr = 0.0002
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

In [ ]:
#训练网络
for epoch in range(50):
    for i, (images, _) in enumerate(train_loader):
        images = Variable(images)
        real_labels = Variable(torch.ones(images.size(0)))
        fake_labels = Variable(torch.zeros(images.size(0)))
        
        # Train the discriminator
        discriminator.zero_grad()
        outputs = discriminator(images)
        real_loss = criterion(outputs, real_labels)
        real_score = outputs
        
        noise = Variable(torch.randn(images.size(0), 128))
        fake_images = generator(noise)
        outputs = discriminator(fake_images.detach()) 
        fake_loss = criterion(outputs, fake_labels)
        fake_score = outputs
        
        d_loss = real_loss + fake_loss
        d_loss.backward()
        d_optimizer.step()
        
        # Train the generator 
        generator.zero_grad()
        noise = Variable(torch.randn(images.size(0), 128))
        fake_images = generator(noise)
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, g_loss: %.4f, ' 
                  'D(x): %.2f, D(G(z)): %.2f' 
                  %(epoch, 50, i+1, 500, d_loss.data[0], g_loss.data[0],
                    real_score.data.mean(), fake_score.data.mean()))
            
            # Save the sampled images
            torchvision.utils.save_image(fake_images.data, 
                './same/fake_samples_%d_%d.png' %(epoch+1, i+1))

In [164]:
#保存模型
torch.save(generator.state_dict(), './generator.pkl')
torch.save(discriminator.state_dict(), './discriminator.pkl')